In [486]:

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt 
import seaborn as sns
import requests
import json
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import spacy

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
# conda install -c conda-forge webdriver-manager
# python -m spacy download en_core_web_lg
# conda install -c conda-forge distance
# conda install -c anaconda xlrd

from datetime import date

today = date.today()

In [487]:
str(today)

'2022-12-23'

# Load DataSet and Data Dictionary

In [488]:
# main_dir = 
Dataset_name = 'TechLabsDataset.csv' 
DataDictionary_name = 'TechLabsDataset_Dictionary.csv'

Dataset = pd.read_csv(os.path.join('./Data/',Dataset_name), index_col = 0)
DataDictionary = pd.read_csv(os.path.join('./Data/',DataDictionary_name), index_col = 0)
Dataset.head()

,name,type_company,lat,lon,city,avg_stars_num,n_reviews_num,encoded_user,local_guide,clean_other_review_num,published_date,today_date,stars_num,review_EN,original_lang
0,Caretrex warehousing & logistics,Logistics service,51.593721,5.073492,Tilburg,5.0,2.0,HHpDM,0.0,4.0,2 years ago,2022-09-24,5.0,NaN,NaN
1,Caretrex warehousing & logistics,Logistics service,51.593721,5.073492,Tilburg,5.0,2.0,dYFWx,0.0,1.0,5 years ago,2022-09-24,5.0,NaN,NaN
2,FEFA Logistics,Trucking company,51.480074,5.446764,Eindhoven,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dekkers Transport Holland,Trucking company,51.597249,5.027990,Tilburg,4.6,53.0,Pp1)c,0.0,5.0,a year ago,2022-09-24,5.0,Best employer ever. Worked there for approxima...,ENG
4,Dekkers Transport Holland,Trucking company,51.597249,5.027990,Tilburg,4.6,53.0,*nk1l,0.0,2.0,a year ago,2022-09-24,5.0,better place with very good people everything ...,ENG


In [489]:
number_reviews_per_company = Dataset[~Dataset['review_EN'].isnull()][['name','city','type_company']].value_counts().reset_index(name ='#reviews')

# number_reviews_per_company['#reviews'].describe()
# number_reviews_per_company[number_reviews_per_company['#reviews']>=20]
number_reviews_per_company

,name,city,type_company,#reviews
0,First Logistics B.V.,Nieuwegein,Trucking company,293
1,Van Spreuwel Transport,Eindhoven,Trucking company,288
2,NUNNER Logistics BV,Helmond,Trucking company,143
3,Claassen Logistics,Tilburg,Trucking company,135
4,ID Logistics Benelux,Tilburg,Logistics service,123
...,...,...,...,...
183,STL Logistics B.V.,Barendrecht,Logistics,1
184,Global Logistics Holland B.V.,Hedel,Trucking company,1
185,Frankena Logistics,Spijkenisse,Trucking company,1
186,VDR logistics,IJssel,Trucking company,1


# scrape from https://graydongo.nl/nl/

In [307]:


# set driver options


def get_graydongo(company_name,company_city):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    url = 'https://graydongo.nl/nl/'
    driver.get(url)

    time.sleep(4)
    # click on cookies
    cook_btn = driver.find_element(By.ID, 'CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll')
    cook_btn.click()

    # set company name in search box
    text_space = driver.find_element(By.NAME, "search")
    text_space.send_keys(company_name)
    time.sleep(4)

    # click on search box
    cook_btn =  driver.find_element(By.CSS_SELECTOR, "button[type='button']")
    cook_btn.click()
    time.sleep(10)

    # alternative - 
    cook_btn = driver.find_elements(By.CLASS_NAME, "company-name")


    get_all_companies_in_search_list = list()
    for j in np.arange(len(cook_btn)):
        get_all_companies_in_search_list.append(cook_btn[j].text)
    
    # click on the first company
    html_str = cook_btn[0].get_attribute('innerHTML')
    new_url = html_str.split('"')[1]
    
    driver.get(new_url)

    # # alternative - 
    # cook_btn =  driver.find_elements(By.CLASS_NAME, "search-highlight")

    resp = BeautifulSoup(driver.page_source, 'html.parser')

    GD_Name = resp.find_all('h1', class_='h2')[0].text.replace('\n','')
    GD_Med = resp.find_all('div', class_='company-text__highlight-label')[0].text.replace('\n','')
    GD_Act = resp.find_all('div', class_='company-text__highlight-label')[1].text.replace('\n','')
    GD_City = resp.find_all('div', class_='company-text__highlight-label')[2].text.replace('\n','')
    GD_kvk_ref = resp.find_all('div', class_='company-text__highlight-label')[3].text.replace('\n','')
    GD_other_in_list = get_all_companies_in_search_list
    
    
    
    time.sleep(4)
    
    driver.close()
    driver.quit()
    
    return [company_name, company_city, GD_Name,GD_Med,GD_Act,GD_City,GD_kvk_ref,GD_other_in_list]


In [523]:
GD_list = list()
for i in np.arange(len(number_reviews_per_company.iloc[i]['name'])):
    #for each company scrape data on graydongo
    company_name = number_reviews_per_company.iloc[i]['name']
    company_city = number_reviews_per_company.iloc[i]['city']
    GD_list.append(get_graydongo(company_name,company_city))

In [318]:
# convert list to dataframe
GD_list_df = pd.DataFrame(GD_list, columns = ['name', 'city', 'GD_Name','GD_Med','GD_Act','GD_City','GD_kvk_ref','GD_other_in_list'])
GD_list_df.shape

(187, 8)

In [316]:
# save scraped data
GD_list_df.to_csv('./output/scaping_graydongo_'+str(today)+'.csv')

# process data scraped from https://graydongo.nl/nl/

In [484]:

# GD_list_df = pd.read_csv('./output/scaping_graydongo.csv', index_col = 0)
GD_list_df = pd.read_excel('./output/scaping_graydongo_2022-12-16_checked.xls', index_col = 0)

In [485]:
GD_list_df[GD_list_df['name']=='Chain Logistics BV']

,name,city,same_name,GD_Name,GD_Med,GD_Act,GD_City,GD_kvk_ref,GD_other_in_list
60,Chain Logistics BV,Uden,1,Chain Logistics B.V. ...,36 werknemers,Actief sinds 2000,UDEN ...,"52291 - Expediteurs, cargadoors, bevrachters e...","['Chain Logistics B.V.', 'Fresh Chain Logistic..."


In [470]:
GD_list_df.head()

,name,city,same_name,GD_Name,GD_Med,GD_Act,GD_City,GD_kvk_ref,GD_other_in_list
0,First Logistics B.V.,Nieuwegein,1,First Logistics B.V. ...,27 werknemers,Actief sinds 2018,NIEUWEGEIN ...,4941 - Goederenvervoer over de weg (geen verhu...,"['First Logistics B.V.', 'First Logistics B.V...."
1,Van Spreuwel Transport,Eindhoven,1,Van Spreuwel Transport B.V...,118 werknemers,Actief sinds 1997,EINDHOVEN ...,4941 - Goederenvervoer over de weg (geen verhu...,"['Van Spreuwel Transport B.V.', 'van Spreuwel ..."
2,NUNNER Logistics BV,Helmond,1,Nunner Logistics B.V. ...,162 werknemers,Actief sinds 1989,HELMOND ...,4941 - Goederenvervoer over de weg (geen verhu...,"['Nunner Logistics B.V.', 'NUNNER Logistics WH..."
3,Claassen Logistics,Tilburg,1,Claassen Logistics Meppel ...,22 werknemers,Actief sinds 2012,ROGAT ...,4941 - Goederenvervoer over de weg (geen verhu...,['Claassen Logistics Meppel B.V.']
4,ID Logistics Benelux,Tilburg,1,ID Logistics Benelux B.V. ...,395 werknemers,Actief sinds 2011,TILBURG ...,52109 - Opslag in distributiecentra en overige...,"['ID Logistics Benelux B.V.', 'ID Logistics Be..."


In [471]:
GD_list_df['same_name'].value_counts(dropna = False)

1    171
0     16
Name: same_name, dtype: int64

In [472]:
# GD_list_df['GD_Med'].value_counts(dropna = False)

In [473]:

def get_n_employees(x):
    if 'Onbekend' in x:
        x = np.nan
    else:
        x = float(x.split(' ')[0])
    return x
GD_list_df['#emploeeys'] = GD_list_df['GD_Med'].apply(lambda x: get_n_employees(x))

In [474]:
GD_list_df['company_size'] = ''
GD_list_df.loc[((GD_list_df['#emploeeys']<=250) & (GD_list_df['#emploeeys']>=10)),'company_size']='SME'
GD_list_df.loc[(GD_list_df['#emploeeys']<10),'company_size']='Micro'
GD_list_df.loc[(GD_list_df['#emploeeys']>250),'company_size']='Large'
GD_list_df.loc[GD_list_df['#emploeeys'].isna(),'company_size']='Unknown'
GD_list_df['company_size'].value_counts()

Micro      87
SME        68
Unknown    26
Large       6
Name: company_size, dtype: int64

In [475]:
GD_list_df[['same_name','company_size']].value_counts(dropna = False)

same_name  company_size
1          Micro           79
           SME             66
           Unknown         20
0          Micro            8
           Unknown          6
1          Large            6
0          SME              2
dtype: int64

In [478]:
list_to_select_df = number_reviews_per_company.merge(GD_list_df, on=['name','city'])

list_to_select_df[(list_to_select_df['#reviews']>20) & 
                 (list_to_select_df['same_name']==1) &
                 (list_to_select_df['company_size']=='SME') ]['type_company'].value_counts(dropna = False)




Trucking company              9
Logistics service             6
Freight forwarding service    2
Logistics                     1
Distribution service          1
Storage facility              1
Name: type_company, dtype: int64

In [479]:
list_to_select_df[(list_to_select_df['#reviews']>20) & 
                 (list_to_select_df['same_name']==1) &
                 (list_to_select_df['company_size']=='SME') ] \
                [['name','city','#reviews',
                  '#emploeeys','type_company']].sort_values(['#emploeeys','#reviews'], ascending = False)




,name,city,#reviews,#emploeeys,type_company
2,NUNNER Logistics BV,Helmond,143,162.0,Trucking company
24,Base Logistics BV,Klundert,35,130.0,Freight forwarding service
15,De Graaf Logistics BV,Oosterhout,59,129.0,Distribution service
1,Van Spreuwel Transport,Eindhoven,288,118.0,Trucking company
5,GVT Transport & Logistics Tilburg,Tilburg,117,113.0,Logistics
31,Rhenus Contract Logistics Eindhoven B.V.,Son,25,100.0,Logistics service
22,Van den Broek Logistics BV,Helmond,39,91.0,Trucking company
17,Transport J.A. van Wanrooy BV,Oosterhout,48,70.0,Logistics service
33,Speedliner Logistics BV,Tilburg,23,58.0,Logistics service
30,Schavemaker Transport Breda B.V.,Breda,26,46.0,Trucking company
